**Table of contents**<a id='toc0_'></a>    
- [Feature Engineering](#toc1_)    
  - [데이터 합치기](#toc1_1_)    
  - [필요없는 피처 제거](#toc1_2_)    
  - [데이터 스케일링](#toc1_3_)    
  - [데이터 나누기](#toc1_4_)    
- [모델 훈련 및 성능 검증](#toc2_)    
  - [test dataset 예측 결과](#toc2_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

train_path = './data/train/'
test_path = './data/validation/'

train_label = pd.read_csv(train_path + 'label/1.걸음걸이/training_label.csv')
train = pd.read_csv(train_path + 'raw/train_sleep.csv')

test_label = pd.read_csv(test_path + 'label/1.걸음걸이/val_label.csv')
test = pd.read_csv(test_path + 'raw/val_sleep.csv')

In [16]:
label_encoder = {
    'CN': 0, 
    'MCI': 1,
    'Dem': 2
}

In [17]:
train_label['label'] = train_label['DIAG_NM'].map(label_encoder)
test_label['label'] = test_label['DIAG_NM'].map(label_encoder)
train_label.drop(columns=['DIAG_NM'], inplace=True)
test_label.drop(columns=['DIAG_NM'], inplace=True)

In [ ]:
# label 과 합치기
train = train.merge(train_label, left_on='EMAIL', right_on='SAMPLE_EMAIL')
test = test.merge(test_label, left_on='EMAIL', right_on='SAMPLE_EMAIL')

train.drop(columns=['SAMPLE_EMAIL'], inplace=True)
test.drop(columns=['SAMPLE_EMAIL'], inplace=True)

# <a id='toc1_'></a>[Feature Engineering](#toc0_)

## <a id='toc1_1_'></a>[데이터 합치기](#toc0_)

In [19]:
all_data = pd.concat([train, test], axis=0)
all_data = all_data.drop(['label'], axis=1) # 타겟값 제거

In [ ]:
all_features = all_data.columns
all_features

Index(['EMAIL', 'sleep_awake', 'sleep_bedtime_end', 'sleep_bedtime_start',
       'sleep_breath_average', 'sleep_deep', 'sleep_duration',
       'sleep_efficiency', 'sleep_hr_5min', 'sleep_hr_average',
       'sleep_hr_lowest', 'sleep_hypnogram_5min', 'sleep_is_longest',
       'sleep_light', 'sleep_midpoint_at_delta', 'sleep_midpoint_time',
       'sleep_onset_latency', 'sleep_period_id', 'sleep_rem', 'sleep_restless',
       'sleep_rmssd', 'sleep_rmssd_5min', 'sleep_score',
       'sleep_score_alignment', 'sleep_score_deep', 'sleep_score_disturbances',
       'sleep_score_efficiency', 'sleep_score_latency', 'sleep_score_rem',
       'sleep_score_total', 'sleep_temperature_delta',
       'sleep_temperature_deviation', 'sleep_total',
       'CONVERT(sleep_hr_5min USING utf8)',
       'CONVERT(sleep_hypnogram_5min USING utf8)',
       'CONVERT(sleep_rmssd_5min USING utf8)'],
      dtype='object')

## <a id='toc1_2_'></a>[필요없는 피처 제거](#toc0_)

In [ ]:
drop_features = ['sleep_hr_5min', 'sleep_rmssd_5min', 'sleep_hypnogram_5min',
                 'sleep_period_id', 'sleep_is_longest', 'EMAIL', 'sleep_is_longest', 
                 'sleep_rmssd', 'sleep_score', 'sleep_score_alignment', 'sleep_temperature_delta',
                 'sleep_temperature_deviation'] # drop 확정

remaining_features = [feature for feature in all_features 
                      if feature not in drop_features]

all_data = all_data[remaining_features]
all_data = all_data.select_dtypes(include=[int, float]) # 숫자형 데이터만 

## <a id='toc1_3_'></a>[데이터 스케일링](#toc0_)

In [ ]:
from sklearn.preprocessing import StandardScaler

all_data = StandardScaler().fit_transform(all_data)

## <a id='toc1_4_'></a>[데이터 나누기](#toc0_)

In [ ]:
num_train = len(train)

X = all_data[:num_train]
X_test = all_data[num_train:]

y = train['label'].values
y_test = test['label'].values

# <a id='toc2_'></a>[모델 훈련 및 성능 검증](#toc0_)

In [24]:
random_seed = 24
n_splits = 5

In [25]:
from sklearn.model_selection import StratifiedKFold

# shuffle: 훈련 데이터가 시계열 데이터가 아니라면 섞어주는 것이 좋음
# 시계열 데이터는 순서가 중요해서 데이터를 섞으면 안도미
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb

# 모델 학습 및 평가
models = {
    "Lasso Regression": LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr'),
    "Decision Tree": DecisionTreeClassifier(random_state=random_seed),
    "Support Vector Machine": SVC(kernel='linear', probability=True, random_state=random_seed),
    "Gradient Boosting": GradientBoostingClassifier(random_state=random_seed),
    "LightGBM": lgb.LGBMClassifier(random_state=random_seed, force_col_wise=True) #force_col_wise param: warning 제거를 위한 추가  
}

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

models_ = {}

for name, model in models.items():
    print('#'*40, f"Model: {name}", '#'*40)
    
    acc_scores = []
    f1_scores = []
    class_report_list = []
    confusion_matrices = []
    
    for fold, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        # 데이터 분할
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # 모델 학습
        model.fit(X_train, y_train)
        
        # 모델 예측
        y_pred = model.predict(X_val)
        
        # 평가
        acc = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average='weighted')  # 불균형 데이터 고려
        acc_scores.append(acc)
        f1_scores.append(f1)

        # Classification Report 저장
        report = classification_report(y_val, y_pred, output_dict=True)
        class_report_list.append(pd.DataFrame(report).T)

    # Classification Report 평균
    avg_class_report = pd.concat(class_report_list).groupby(level=0).mean()

    # Model 저장
    models_[name] = model
    
    # Accuracy 및 F1-score 평균 출력
    print(f"=== Average Classification Report ===")
    print(avg_class_report)
    print(f"Average Accuracy: {np.mean(acc_scores):.4f}")
    print(f"Average F1-score: {np.mean(f1_scores):.4f}")
    print("=" * 60)

######################################## Model: Lasso Regression ########################################
=== Average Classification Report ===
              precision    recall  f1-score      support
0              0.612399  0.939282  0.741402  1156.200000
1              0.461160  0.094837  0.157230   670.600000
2              0.461978  0.120839  0.190855   114.200000
accuracy       0.599382  0.599382  0.599382     0.599382
macro avg      0.511846  0.384986  0.363162  1941.000000
weighted avg   0.551308  0.599382  0.507187  1941.000000
Average Accuracy: 0.5994
Average F1-score: 0.5072
######################################## Model: Decision Tree ########################################
=== Average Classification Report ===
              precision    recall  f1-score      support
0              0.732096  0.719771  0.725785  1156.200000
1              0.551775  0.560090  0.555795   670.600000
2              0.400932  0.430664  0.414080   114.200000
accuracy       0.647604  0.647604  0.6

/usr/local/anaconda3/envs/env2501/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/env2501/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/env2501/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

=== Average Classification Report ===
              precision    recall  f1-score      support
0              0.595672  1.000000  0.746610  1156.200000
1              0.000000  0.000000  0.000000   670.600000
2              0.000000  0.000000  0.000000   114.200000
accuracy       0.595672  0.595672  0.595672     0.595672
macro avg      0.198557  0.333333  0.248870  1941.000000
weighted avg   0.354826  0.595672  0.444735  1941.000000
Average Accuracy: 0.5957
Average F1-score: 0.4447
######################################## Model: Gradient Boosting ########################################
=== Average Classification Report ===
              precision    recall  f1-score      support
0              0.698728  0.891367  0.783367  1156.200000
1              0.680108  0.416644  0.516597   670.600000
2              0.802444  0.385370  0.519300   114.200000
accuracy       0.697579  0.697579  0.697579     0.697579
macro avg      0.727093  0.564460  0.606421  1941.000000
weighted avg   0.698400  0

## <a id='toc2_1_'></a>[test dataset 예측 결과](#toc0_)

In [36]:
# Test Set 성능 평가
print("\n", "="*40, "TEST SET EVALUATION", "="*40, "\n")

for name, model in models_.items():
    print(f"=== Evaluating {name} on Test Set ===")

    # 테스트 데이터 예측
    y_test_pred = model.predict(X_test)

    # 테스트 데이터 평가
    test_acc = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')  # 불균형 고려
    test_report = classification_report(y_test, y_test_pred, output_dict=True)
    test_conf_matrix = confusion_matrix(y_test, y_test_pred)

    # 결과 출력
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test F1-score: {test_f1:.4f}")
    print(f"Test Classification Report:\n", pd.DataFrame(test_report).T)
    print("=" * 60)


 ======================================== TEST SET EVALUATION ======================================== 

=== Evaluating Lasso Regression on Test Set ===
Test Accuracy: 0.7510
Test F1-score: 0.6922
Test Classification Report:
               precision    recall  f1-score      support
0              0.796376  0.943763  0.863828  1956.000000
1              0.033784  0.016234  0.021930   308.000000
2              0.833333  0.046729  0.088496   214.000000
accuracy       0.751009  0.751009  0.751009     0.751009
macro avg      0.554498  0.335575  0.324751  2478.000000
weighted avg   0.704782  0.751009  0.692227  2478.000000
=== Evaluating Decision Tree on Test Set ===
Test Accuracy: 0.5589
Test F1-score: 0.6013
Test Classification Report:
               precision    recall  f1-score      support
0              0.813151  0.638548  0.715349  1956.000000
1              0.134503  0.373377  0.197764   308.000000
2              0.241379  0.098131  0.139535   214.000000
accuracy       0.558918  0.5

/usr/local/anaconda3/envs/env2501/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/env2501/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/env2501/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)